In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import os.path
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import Birch
import  glob
from pprint import pprint
import pickle

In [2]:
Doc = collections.namedtuple('Doc',['investigator', 'amount', 'abstract'])

def load2(fn):
    doc = nltk.data.load(fn)
    wrds = nltk.tokenize.wordpunct_tokenize(doc)
    investigator = " ".join(wrds[wrds.index("Investigator")+2:
                          min(wrds.index('@' if '@' in wrds else "Abstract", 
                                          wrds.index("Investigator"))-1, 
                              wrds.index('(', wrds.index("Investigator")))])
    amount = int(wrds[wrds.index("Amt") + 4: wrds.index('(', wrds.index("Amt"))][0])
    abstract = " ".join(wrds[wrds.index('Abstract')+2:])
    return Doc(investigator, amount, abstract)

In [4]:
filenames = [fn for fn in glob.iglob('./text/Part*/*/*/*.txt', recursive=False)]
words = []
for i in range(4000):
    words += load2(filenames[i]).abstract.split(" ")

In [5]:
w_vocab = np.unique(words)

In [6]:
len(w_vocab)

31850

In [7]:
vocabulary_size = 31850

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
#del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:20])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 1], ('the', 39472), ('of', 37005), ('.', 29906), ('and', 26950), (',', 25266), ('to', 17952), ('in', 15637), ('will', 10950), ('a', 10623), ('-', 10106), ('The', 8042), ('is', 7897), ('be', 7580), ('for', 7342), ('on', 5197), ('(', 4685), ('are', 4335), ('that', 4277), (')', 4235)]
Sample data [50, 1022, 1851, 5, 169, 5451, 2, 255, 8419, 142] ['In', 'tropical', 'forests', ',', 'most', 'colonization', 'of', 'small', 'disturbed', 'areas']


In [8]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(data, batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(data, batch_size=8, num_skips=2, skip_window=3)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 240
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 6       # How many words to consider left and right.
num_skips = 6         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 100    # Number of negative examples to sample.

5 , -> 50 In
5 , -> 1022 tropical
169 most -> 5 ,
169 most -> 2 of
5451 colonization -> 169 most
5451 colonization -> 8419 disturbed
2 of -> 5451 colonization
2 of -> 8419 disturbed


In [9]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 20001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(data,
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
                
        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print("Average loss at step ", step, ": ", average_loss)
          average_loss = 0
    final_embeddings = normalized_embeddings.eval()
    

Average loss at step  0 :  370.46395874
Average loss at step  2000 :  108.538758676
Average loss at step  4000 :  33.862258956
Average loss at step  6000 :  18.9935306528
Average loss at step  8000 :  13.6198228078
Average loss at step  10000 :  11.5406381334
Average loss at step  12000 :  11.1455249362
Average loss at step  14000 :  10.168638386
Average loss at step  16000 :  7.31043463302
Average loss at step  18000 :  5.58411948538
Average loss at step  20000 :  5.56684179831


Reduce Vocab to 5000 Groups

In [46]:
embd = np.array([final_embeddings[datum] for datum in data])
fit = MiniBatchKMeans(n_clusters=5000, max_iter=200000, batch_size=2000)
labels = fit.fit(embd).labels_
vocabulary_size = fit.n_clusters

Setup Tree for Recovering Words

In [12]:
reverse_dictionary[0] = 'UNK'

In [45]:
network = nx.DiGraph()

In [46]:
for word, number in dictionary.items():
    network.add_node(number, word=word)

In [30]:
reverse_dictionary[0] = 'UNK'
labels = data

In [31]:
gr_dict = {}
rev_gr_dict = {}
for index in range(len(data)):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if reverse_dictionary[data[index]] not in gr_dict:
        gr_dict[reverse_dictionary[data[index]]] = labels[index]
    rev_gr_dict[labels[index]].update([reverse_dictionary[data[index]]])

Fit First Transitions

## New Stuff Testing 

In [44]:
nl_words2 = [data[0]]
for i in range(len(data[:-1])):
    nl_words2.extend([gr_dict[tuple(data[i:i+2])], data[i+1]])

In [48]:
nl_words3 = [(nl_words2[i], nl_words2[i+1]) for i in range(len(nl_words2)-1)]

In [ ]:
fit = MiniBatchKMeans(n_clusters=2000, max_iter=200000, batch_size=1000)
nl_words = [(labels[i], labels[i+1]) for i in range(len(labels)-1)]
embd = np.array([final_embeddings[datum[1]] - \
                 final_embeddings[datum[0]] for datum in nl_words])
fit = fit.fit(embd)
labels = fit.labels_ + vocabulary_size
vocabulary_size += fit.n_clusters

## Old STUFF

In [15]:
labels = data

In [17]:
fit = MiniBatchKMeans(n_clusters=5000, max_iter=200000, batch_size=5000)
nl_words = [(labels[i], labels[i+1]) for i in range(len(labels)-1)]
embd = np.array([final_embeddings[datum[1]] - \
                 final_embeddings[datum[0]] for datum in nl_words])
fit = fit.fit(embd)
labels = fit.labels_ + vocabulary_size
vocabulary_size += fit.n_clusters

In [47]:
for index in range(len(labels)):
    if labels[index] not in network:
        network.add_node(labels[index], counts=collections.Counter())
    if network.has_edge(nl_words[index][0],nl_words[index][1]):
        network[nl_words[index][0]][nl_words[index][1]]['weight'] += 1
    else:
        network.add_edge(nl_words[index][0],
                    nl_words[index][1], weight=1)
    network.node[labels[index]]['counts'].update([
        (nl_words[index][0], nl_words[index][1])])
    

In [34]:
for index in range(len(nl_words[:-1])):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if nl_words[index] not in gr_dict:
        gr_dict[nl_words[index]] = labels[index]
    rev_gr_dict[labels[index]].update([nl_words[index]])

Run Loop To Make Levels

In [39]:
nnl_data = data

In [98]:
nl_data = [nnl_data[0]]

for i in range(len(nl_words)):
    nl_data.extend([labels[i], nnl_data[i+1]])

nnl_data = labels

In [99]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 20001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(nl_data,
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print("Average loss at step ", step, ": ", average_loss)
          average_loss = 0
    final_embeddings = normalized_embeddings.eval()

Average loss at step  0 :  462.020568848
Average loss at step  2000 :  154.41441855
Average loss at step  4000 :  52.3635837234
Average loss at step  6000 :  26.7389970212
Average loss at step  8000 :  16.4095976932
Average loss at step  10000 :  10.026903705
Average loss at step  12000 :  7.05871009189
Average loss at step  14000 :  5.19008916959
Average loss at step  16000 :  4.1020967851
Average loss at step  18000 :  3.26669571501
Average loss at step  20000 :  2.89260978785


In [94]:
fit.set_params(n_clusters=2000, batch_size=1000)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=200000, max_no_improvement=10,
        n_clusters=2000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)

Record all the lower level nodes which make up transitions and fit the

In [95]:
nl_words = [(labels[i], labels[i+1]) for i in range(len(labels)-1)]
embd_l = np.array([final_embeddings[datum] for datum in nnl_data])
fit = fit.fit(embd_l[1:] - embd_l[:-1])

In [96]:
labels = fit.labels_ + vocabulary_size
vocabulary_size += fit.n_clusters
for index in range(len(nl_words[:-1])):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if nl_words[index] not in gr_dict:
        gr_dict[nl_words[index]] = labels[index]
    rev_gr_dict[labels[index]].update([nl_words[index]])
    

In [97]:
for index in range(len(labels)):
    if labels[index] not in network:
        network.add_node(labels[index], counts=collections.Counter())
    if network.has_edge(nl_words[index][0],nl_words[index][1]):
        network[nl_words[index][0]][nl_words[index][1]]['weight'] += 1
    else:
        network.add_edge(nl_words[index][0],
                    nl_words[index][1], weight=1)
    network.node[labels[index]]['counts'].update([
        (nl_words[index][0], nl_words[index][1])])    

Sample the Tree

In [77]:
test = collections.Counter(nl_data).most_common(50)[0][0]
samp = sample(test, rev_gr_dict)
for pre, _, node in anytree.RenderTree(samp):
    print("%s%s" % (pre, node.name))

46958
├── 45119
│   ├── 41652
│   │   ├── 36082
│   │   │   ├── 424
│   │   │   │   └── They
│   │   │   └── 8
│   │   │       └── will
│   │   └── 32419
│   │       ├── 955
│   │       │   └── extremely
│   │       └── 45
│   │           └── important
│   └── 40770
│       ├── 31862
│       │   ├── 24775
│       │   │   └── undetectable
│       │   └── 7
│       │       └── in
│       └── 32899
│           ├── 2
│           │   └── of
│           └── 391
│               └── Chemistry
└── 44390
    ├── 39007
    │   ├── 32419
    │   │   ├── 13
    │   │   │   └── be
    │   │   └── 45
    │   │       └── important
    │   └── 34586
    │       ├── 45
    │       │   └── important
    │       └── 641
    │           └── implications
    └── 40377
        ├── 34586
        │   ├── 45
        │   │   └── important
        │   └── 209
        │       └── because
        └── 32540
            ├── 18
            │   └── that
            └── 565
                └── often


In [74]:
from anytree.dotexport import RenderTreeGraph
tree = RenderTreeGraph(samp)
tree.to_picture('./test14.png')

In [67]:
import anytree
def sample(label, top, parent=None):
    if not(type(label) == str) and label > 3719:
        node = anytree.Node(label, parent=parent)
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        choices = list(top[label].keys())
        if len(choices) == 1 and type(choices[0]) == str:
            anytree.Node(choices[0], parent=node)
        else:
            indices = [i for i in range(len(choices))]
            choice = choices[np.random.choice(indices, p=probabilities)]
            sample(choice[0], top, node)
            sample(choice[1], top, node)
    else:
        parent = anytree.Node(label, parent=parent)
        probabilities = list(top[label].values())
        if len(probabilities) > 1:
            total = sum(list(top[label].values()))
            probabilities = np.array(probabilities)/total
            choices = list(top[label].keys())
            indices = [i for i in range(len(choices))]
            choice = choices[np.random.choice(indices, p=probabilities)]
            node = anytree.Node(choice, parent=parent)
        else:
            node = anytree.Node(list(top[label].keys())[0], parent=parent)
    
    return node

In [68]:
def sample_sent(label, top):
    if type(list(top[label].keys())[0]) == tuple:
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        choices = list(top[label].keys())
        indices = [i for i in range(len(choices))]
        choice = choices[np.random.choice(indices, p=probabilities)]
        l_sent = sample_sent(choice[0], top)
        r_sent = sample_sent(choice[1], top)
        
        return l_sent +  r_sent
    else:
        return [label]

In [100]:
revdic = open('rev_dic.pkl', 'wb')

pickle.dump(rev_gr_dict, revdic, protocol=3)

revdic.close()
dic = open('dic.pkl', 'wb')
pickle.dump(gr_dict, dic, protocol=3)

np.save('./fembd.npy', final_embeddings)
nx.write_gpickle(network, './network.gpickle')

Tree Training

In [101]:
dc = open('./dic.pkl','rb')
rdc = open('./rev_dic.pkl', 'rb')
bottom = pickle.load(dc, )
top = pickle.load(rdc)
dc.close()
rdc.close()


In [58]:
for i in range(len(nnl_data)):
    nnnl_data.extend([nnl_data[i], nl_data[i+1], data[i+1]])

In [113]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 20001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(nnnl_data,
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print("Average loss at step ", step, ": ", average_loss)
          average_loss = 0
    final_embeddings = normalized_embeddings.eval()

Average loss at step  0 :  359.297271729
Average loss at step  2000 :  73.6223668047
Average loss at step  4000 :  10.6700518448
Average loss at step  6000 :  6.15450251514
Average loss at step  8000 :  5.11485475594
Average loss at step  10000 :  4.7450978049
Average loss at step  12000 :  4.38066986352
Average loss at step  14000 :  4.09571653277
Average loss at step  16000 :  3.95512743896
Average loss at step  18000 :  3.54849309093
Average loss at step  20000 :  3.20592445511


In [110]:
nnnl_words = [(nnnl_data[i], nnnl_data[i+1]) for i in range(len(nnnl_data)-1)]
embd_l = np.array([final_embeddings[datum] for datum in nnnl_data])
fit = fit.fit(embd_l[1:] - embd_l[:-1])

In [111]:
labels = fit.labels_ + vocabulary_size
vocabulary_size += fit.n_clusters
for index in range(len(nl_words[:-1])):
    if labels[index] not in rev_gr_dict:
        rev_gr_dict[labels[index]] = collections.Counter()
    if nl_words[index] not in gr_dict:
        gr_dict[nl_words[index]] = labels[index]
    rev_gr_dict[labels[index]].update([nl_words[index]])
    

In [112]:
nnnl_data = [nnnl_words[0][0]]

for i in range(len(labels)):
    nnnl_data.extend([labels[i], nnnl_words[i][1]])

nnnnl_data = labels